<a href="https://colab.research.google.com/github/vincent4u/CE807_Text_Analytics/blob/main/Word2Vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install gensim

In [2]:
!pip install python-Levenshtein

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.4/177.4 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 12.0 MB/s eta 0:00:00


In [5]:
import gensim
import pandas as pd
import numpy as np

import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

student_id = 2314555
np.random.seed(student_id)


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [12]:
train_df = pd.read_csv('/content/train.csv')
#train_df.head(20)
val_df=pd.read_csv('/content/valid.csv')
#val_df.head(20)
test_df=pd.read_csv('/content/test.csv')
#test_df.head(20)
print(train_df.columns)
train_df['comment'].head()

Index(['comment_id', 'comment', 'split', 'toxicity'], dtype='object')


0    SDATA_5 :  NEWLINE_TOKENNEWLINE_TOKEN== Kinder...
1    SDATA_5 :  NEWLINE_TOKENNEWLINE_TOKENDo not vi...
2    SDATA_5 :  NEWLINE_TOKEN:::Judging by some of ...
3          SDATA_5 :  I LOVE FATTY SEX MMMM  : EDATA_5
4    SDATA_5 :  NEWLINE_TOKEN:Please, could you exp...
Name: comment, dtype: object

In [17]:
import re
from tqdm.auto import tqdm

def preprocess_text(text):
    # Remove specific tokens
    text = text.replace("SDATA_5 :", "").replace("EDATA_5", "").replace("NEWLINE_TOKEN", "")


    # Remove special characters and punctuation marks using regular expressions
    text = re.sub(r'[^\w\s]', '', text)

    # Remove URLs, email addresses, and phone numbers
    text = re.sub(r"http\S+|www\S+|\S+@\S+|\d{10}", "", text)

    # Remove numbers
    text = re.sub(r"\d+", "", text)

    # Remove non-ASCII characters
    text = text.encode("ascii", "ignore").decode()

    # Convert the text to lowercase
    text = text.lower()

    # Remove extra white spaces
    preprocessed_text = re.sub(r"\s+", " ", text)

    return preprocessed_text

# Preprocess the text
train_df['preprocessed_comment'] = train_df['comment'].apply(preprocess_text)
val_df['preprocessed_comment'] = val_df['comment'].apply(preprocess_text)



tqdm.pandas()  # Enable progress_apply for pandas
train_df['preprocessed_comment'] = train_df['comment'].progress_apply(preprocess_text)
val_df['preprocessed_comment'] = val_df['comment'].progress_apply(preprocess_text)


val_df['preprocessed_comment'].head()

  0%|          | 0/8699 [00:00<?, ?it/s]

  0%|          | 0/2920 [00:00<?, ?it/s]

0     how is mine pov if i have to repost every aut...
1     referencebot skrippner page diff commentquest...
2     i knew you didnt like galleries and thats why...
3     i apologize ive been having a bad few days so...
4     welcomehello and welcome to wikipedia thank y...
Name: preprocessed_comment, dtype: object

In [20]:
training_comment=train_df['preprocessed_comment'].apply(gensim.utils.simple_preprocess)
valid_comment=val_df['preprocessed_comment'].apply(gensim.utils.simple_preprocess)

valid_comment.head()

0    [how, is, mine, pov, if, have, to, repost, eve...
1    [referencebot, skrippner, page, diff, commentq...
2    [knew, you, didnt, like, galleries, and, thats...
3    [apologize, ive, been, having, bad, few, days,...
4    [welcomehello, and, welcome, to, wikipedia, th...
Name: preprocessed_comment, dtype: object

In [23]:
import nltk
from nltk.corpus import stopwords
from gensim.models import Word2Vec

# Download NLTK stopwords if you haven't already
nltk.download('stopwords')

# Get the English stopwords from NLTK
stop_words = set(stopwords.words('english'))

# Filter out the stopwords from your preprocessed text
training_comment_filtered = [[word for word in comment if word not in stop_words] for comment in training_comment]
valid_comment_filtered = [[word for word in comment if word not in stop_words] for comment in valid_comment]


print(valid_comment_filtered)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


[['mine', 'pov', 'repost', 'every', 'authoritarian', 'sources', 'section', 'sothe', 'following', 'style', 'guides', 'popular', 'publications', 'properly', 'address', 'country', 'finally', 'government', 'refers', 'english', 'dont', 'know', 'authoritarian', 'sources', 'would', 'convince', 'everyone', 'proper', 'usage', 'would', 'let', 'know', 'go', 'find', 'otherwise', 'please', 'refrain', 'using', 'defense', 'higher', 'source', 'wont', 'accept', 'even', 'ridiculous', 'able', 'call', 'feel', 'likealso', 'would', 'need', 'shown', 'virtually', 'relevant', 'authorities', 'english', 'usage', 'agreed', 'point', 'much', 'doubt', 'challenge', 'prove', 'otherwise'], ['referencebot', 'skrippner', 'page', 'diff', 'commentquestion', 'someone', 'alias', 'goblin', 'face', 'making', 'unauthorized', 'changes', 'stanley', 'krippner', 'bio', 'pageskrippner'], ['knew', 'didnt', 'like', 'galleries', 'thats', 'asked', 'images', 'best', 'could', 'camera', 'either', 'option', 'full', 'body', 'shot', 'zoomedin

In [27]:
model=gensim.models.Word2Vec(window=10,
                             min_count=2,
                             workers=4)
model.build_vocab(training_comment,progress_per=1000)
model.train(training_comment, total_examples = model.corpus_count, epochs=model.epochs)

(1991831, 2716300)

In [30]:
import pandas as pd
import gensim
from gensim.models import Word2Vec
import numpy as np
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, f1_score,recall_score, precision_score

# Step 1: Prepare your data
train_df = pd.read_csv('/content/train.csv')
val_df=pd.read_csv('/content/valid.csv')

# Preprocess your text data here (tokenization, removing stopwords, etc.)
def preprocess_text(text):
    # Remove specific tokens
    text = text.replace("SDATA_5 :", "").replace("EDATA_5", "").replace("NEWLINE_TOKEN", "")
    # Remove special characters and punctuation marks using regular expressions
    text = re.sub(r'[^\w\s]', '', text)
    # Remove URLs, email addresses, and phone numbers
    text = re.sub(r"http\S+|www\S+|\S+@\S+|\d{10}", "", text)
    # Remove numbers
    text = re.sub(r"\d+", "", text)
    # Remove non-ASCII characters
    text = text.encode("ascii", "ignore").decode()
    # Convert the text to lowercase
    text = text.lower()
    # Remove extra white spaces
    preprocessed_text = re.sub(r"\s+", " ", text)

    return preprocessed_text

# Preprocess the text
train_df['preprocessed_comment'] = train_df['comment'].apply(preprocess_text)
val_df['preprocessed_comment'] = val_df['comment'].apply(preprocess_text)

tqdm.pandas()  # Enable progress_apply for pandas
train_df['preprocessed_comment'] = train_df['comment'].progress_apply(preprocess_text)
val_df['preprocessed_comment'] = val_df['comment'].progress_apply(preprocess_text)


# Split data into features (X) and labels (y)
X_train = train_df['preprocessed_comment']
y_train = train_df['toxicity']
X_val = val_df['preprocessed_comment']
y_val = val_df['toxicity']

# Step 2: Train Word2Vec model
tokenized_train_comments = [gensim.utils.simple_preprocess(comment) for comment in X_train]
word2vec_model = gensim.models.Word2Vec(window=10,
                             min_count=2,
                             workers=4)
word2vec_model.build_vocab(tokenized_train_comments)
word2vec_model.train(tokenized_train_comments, total_examples=word2vec_model.corpus_count, epochs=10)

# Step 3: Transform text data to word embeddings
def comment_to_vector(comment, model):
    vectors = []
    for word in comment:
        if word in model.wv.key_to_index:  # Check if word is in the vocabulary
            vectors.append(model.wv[word])
    if vectors:
        return np.mean(vectors, axis=0)
    else:
        return np.zeros(model.vector_size)

X_train_word_embeddings = np.array([comment_to_vector(comment, word2vec_model) for comment in tokenized_train_comments])

# Step 4: Train XGBoost classifier
xgb_classifier = XGBClassifier()
xgb_classifier.fit(X_train_word_embeddings, y_train)

# Evaluate the model on validation data
tokenized_val_comments = [gensim.utils.simple_preprocess(comment) for comment in X_val]
X_val_word_embeddings = np.array([comment_to_vector(comment, word2vec_model) for comment in tokenized_val_comments])
predictions = xgb_classifier.predict(X_val_word_embeddings)
accuracy = accuracy_score(y_val, predictions)
f1 = f1_score(y_val, predictions)
precision = precision_score(y_val, predictions)
print("Validation Accuracy:", accuracy)
print("Validation f1-score:", f1)

  0%|          | 0/8699 [00:00<?, ?it/s]

  0%|          | 0/2920 [00:00<?, ?it/s]

Validation Accuracy: 0.8613013698630136
Validation f1-score: 0.0


In [33]:
import pandas as pd
import gensim
import numpy as np
from xgboost import XGBClassifier
from sklearn.metrics import f1_score, precision_score, recall_score

# Load and preprocess data
train_df = pd.read_csv('/content/train.csv')
val_df = pd.read_csv('/content/valid.csv')

# Define preprocessing function
def preprocess_text(text):
    # Remove specific tokens
    text = text.replace("SDATA_5 :", "").replace("EDATA_5", "").replace("NEWLINE_TOKEN", "")
    # Remove special characters and punctuation marks using regular expressions
    text = re.sub(r'[^\w\s]', '', text)
    # Remove URLs, email addresses, and phone numbers
    text = re.sub(r"http\S+|www\S+|\S+@\S+|\d{10}", "", text)
    # Remove numbers
    text = re.sub(r"\d+", "", text)
    # Remove non-ASCII characters
    text = text.encode("ascii", "ignore").decode()
    # Convert the text to lowercase
    text = text.lower()
    # Remove extra white spaces
    preprocessed_text = re.sub(r"\s+", " ", text)

    return preprocessed_text

# Preprocess the text
train_df['preprocessed_comment'] = train_df['comment'].apply(preprocess_text)
val_df['preprocessed_comment'] = val_df['comment'].apply(preprocess_text)

tqdm.pandas()  # Enable progress_apply for pandas
train_df['preprocessed_comment'] = train_df['comment'].progress_apply(preprocess_text)
val_df['preprocessed_comment'] = val_df['comment'].progress_apply(preprocess_text)


# Split data into features (X) and labels (y)
X_train = train_df['preprocessed_comment']
y_train = train_df['toxicity']
X_val = val_df['preprocessed_comment']
y_val = val_df['toxicity']

# Train Word2Vec model
tokenized_comments = [gensim.utils.simple_preprocess(comment) for comment in train_df['preprocessed_comment']]
word2vec_model = gensim.models.Word2Vec(sentences=tokenized_comments, vector_size=100, window=5, min_count=1, workers=4)

# Convert comments to word embeddings
def comment_to_vector(comment, model):
    vectors = []
    for word in comment:
        if word in model.wv.key_to_index:
            vectors.append(model.wv[word])
    if vectors:
        return np.mean(vectors, axis=0)
    else:
        return np.zeros(model.vector_size)

X_train = np.array([comment_to_vector(comment, word2vec_model) for comment in tokenized_comments])
y_train = train_df['toxicity']

# Train XGBoost classifier
xgb_classifier = XGBClassifier()
xgb_classifier.fit(X_train, y_train)

# Preprocess validation data
tokenized_val_comments = [gensim.utils.simple_preprocess(comment) for comment in val_df['preprocessed_comment']]
X_val = np.array([comment_to_vector(comment, word2vec_model) for comment in tokenized_val_comments])
y_val = val_df['toxicity']

# Make predictions
predictions = xgb_classifier.predict(X_val)

# Calculate metrics
f1 = f1_score(y_val, predictions)
precision = precision_score(y_val, predictions)
recall = recall_score(y_val, predictions)

# Print metrics
print("F1-score:", f1)
print("Precision:", precision)
print("Recall:", recall)


  0%|          | 0/8699 [00:00<?, ?it/s]

  0%|          | 0/2920 [00:00<?, ?it/s]

F1-score: -0.0196078431372549
Precision: -0.16
Recall: -0.010443864229765013
